In [1]:
import sys 
sys.path.append('..')
from cox.utils import Parameters
from cox.store import Store
from cox.readers import CollectionReader
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import numpy as np
import torch as ch
from torch import Tensor
from torch.distributions.normal import Normal
from torch.distributions.multivariate_normal import MultivariateNormal
from torch.distributions import Uniform
from torch.utils.data import DataLoader
from delphi.stats.truncated_regression import truncated_regression
from delphi.utils.datasets import TruncatedRegressionDataset 
from delphi.oracle import Left, Interval
import datetime

In [2]:
STORE_PATH = '/home/pstefanou/Uniform2'
STORE_TABLE_NAME = 'uniform'

store = Store(STORE_PATH)

store.add_table(STORE_TABLE_NAME, { 
    'known_param_mse': float,
    'unknown_param_mse': float, 
    'ols_param_mse': float,
    'alpha': float, 
    'var': float, 
})

Logging in: /home/pstefanou/Uniform2/fe532651-e43d-4963-800e-c4dbefcb42b7


Experiment Constants

In [3]:
# regression parameters
NUM_SAMPLES, DIMS = 10000, 10
LOWER, UPPER = -1, 1
PREDICATE = Left(Tensor([1.0]))

In [4]:
# perform each experiment a total of 10 times
for iter_ in range(1):        
    for VAR in range(1, 21):
        # generate data
        W = Uniform(LOWER, UPPER).sample(ch.Size([DIMS, 1]))
        W0 = Uniform(LOWER, UPPER).sample(ch.Size([1, 1]))
        X = Uniform(0, 1).sample(ch.Size([NUM_SAMPLES, DIMS]))
        y = X.mm(W) + W0 + Normal(ch.zeros(1), ch.sqrt(Tensor([VAR]))).sample(ch.Size([NUM_SAMPLES]))
        # truncate
        indices = PREDICATE(y).nonzero(as_tuple=False).flatten()
        y_trunc, x_trunc = y[indices], X[indices]

        # empirical linear regression
        reg = LinearRegression() 
        reg.fit(x_trunc, y_trunc)
        emp_var = ch.var(ch.from_numpy(reg.predict(x_trunc)) - y_trunc, dim=0).unsqueeze(0)

        # experiment parameters
        args = Parameters({ 
            'alpha': Tensor([y_trunc.size(0)/NUM_SAMPLES]), 
            'phi': PREDICATE, 
            'epochs': 10,
            'num_workers': 1, 
            'batch_size': 100,
            'bias': True,
            'num_samples': 100,
            'clamp': True, 
            'radius': 5.0, 
            'var_lr': 1e-1,
            'var': emp_var,
            'lr': 1e-1,
            'shuffle': True, 
            'tol': 1e-1,
        })


        # truncated linear regression with known noise variance
        data = TruncatedRegressionDataset(x_trunc, y_trunc, bias=args.bias, unknown=False) # dataset 
        S = DataLoader(data, batch_size=args.batch_size, num_workers=args.num_workers, shuffle=args.shuffle)
        trunc_reg = truncated_regression(phi=args.phi, alpha=args.alpha, epochs=args.epochs, var_lr=args.var_lr, lr=args.lr, num_samples=args.num_samples, var=args.var, device=args.device)
        results = trunc_reg.fit(S)
        w_, w0_ = results.weight.detach().cpu(), results.bias.detach().cpu()

        # truncated linear regression with unknown noise variance
        data = TruncatedRegressionDataset(x_trunc, y_trunc, bias=args.bias, unknown=True) # dataset 
        S = DataLoader(data, batch_size=args.batch_size, num_workers=args.num_workers, shuffle=args.shuffle)
        trunc_reg = truncated_regression(phi=args.phi, alpha=args.alpha, epochs=args.epochs, var_lr=args.var_lr, lr=args.lr, num_samples=args.num_samples, device=args.device)
        results = trunc_reg.fit(S)
        var_ = results.lambda_.inverse().detach().cpu()
        w, w0 = (results.v.detach()*var_).cpu(), (results.bias.detach()*var_).cpu()

        # calculate metrics 
        real_params, ols_params, unknown_params, known_params = ch.cat([W, W0]), ch.cat([data.w, data.w0.unsqueeze(0)]), ch.cat([w, w0]), ch.cat([w_.t(), w0_.unsqueeze(0)])
        unknown_param_mse, ols_param_mse, known_param_mse = mean_squared_error(unknown_params, real_params), mean_squared_error(ols_params, real_params), mean_squared_error(known_params, real_params)

        store[STORE_TABLE_NAME].append_row({ 
            'known_param_mse': known_param_mse,
            'unknown_param_mse': unknown_param_mse,
            'ols_param_mse': ols_param_mse,
            'alpha': float(args.alpha.flatten()),
            'var': float(VAR), 
        })

# close current store
store.close()

Epoch:1 | Score tensor([0.0156, 0.0420, 0.0039, 0.0564, 0.0210, 0.0147, 0.0497, 0.0114, 0.0701,
        0.0863, 0.0660]) 
Epoch:1 | Score tensor([0.0156, 0.0420, 0.0039, 0.0564, 0.0210, 0.0147, 0.0497, 0.0114, 0.0701,
        0.0863, 0.0660]) 
Epoch:1 | Score tensor([0.0156, 0.0420, 0.0039, 0.0564, 0.0210, 0.0147, 0.0497, 0.0114, 0.0701,
        0.0863, 0.0660]) 
 | Loss 0.2793 | ||:   0%|          | 0/62 [00:00<?, ?it/s]
  0%|          | 0/62 [00:00<?, ?it/s]Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f7f21515be0>>
Traceback (most recent call last):
  File "/home/pstefanou/env/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 1203, in __del__
    self._shutdown_workers()
  File "/home/pstefanou/env/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 1177, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python

Epoch:1 | Score tensor([0.0741, 0.0758, 0.0720, 0.0835, 0.0747, 0.0732, 0.0722, 0.0775, 0.0698,
        0.0664, 0.1489]) 
Epoch:1 | Score tensor([0.0762, 0.0773, 0.0737, 0.0841, 0.0763, 0.0747, 0.0734, 0.0788, 0.0719,
        0.0698, 0.1523]) 
Epoch:1 | Score tensor([0.0762, 0.0773, 0.0737, 0.0841, 0.0763, 0.0747, 0.0734, 0.0788, 0.0719,
        0.0698, 0.1523]) 
Epoch:1 | Score tensor([0.0757, 0.0769, 0.0741, 0.0819, 0.0755, 0.0752, 0.0733, 0.0782, 0.0730,
        0.0690, 0.1518]) 
Epoch:1 | Score tensor([0.0757, 0.0769, 0.0741, 0.0819, 0.0755, 0.0752, 0.0733, 0.0782, 0.0730,
        0.0690, 0.1518]) 
Epoch:1 | Score tensor([0.0785, 0.0795, 0.0771, 0.0847, 0.0766, 0.0762, 0.0757, 0.0801, 0.0753,
        0.0729, 0.1561]) 
Epoch:1 | Score tensor([0.0785, 0.0795, 0.0771, 0.0847, 0.0766, 0.0762, 0.0757, 0.0801, 0.0753,
        0.0729, 0.1561]) 
Epoch:1 | Score tensor([0.0785, 0.0784, 0.0757, 0.0835, 0.0766, 0.0773, 0.0750, 0.0794, 0.0738,
        0.0718, 0.1549]) 
Epoch:1 | Score tensor([

Epoch:1 | Score tensor([0.0783, 0.0803, 0.0774, 0.0819, 0.0792, 0.0782, 0.0808, 0.0822, 0.0776,
        0.0786, 0.1610]) 
Epoch:1 | Score tensor([0.0771, 0.0784, 0.0760, 0.0799, 0.0774, 0.0768, 0.0801, 0.0817, 0.0756,
        0.0772, 0.1585]) 
Epoch:1 | Score tensor([0.0771, 0.0784, 0.0760, 0.0799, 0.0774, 0.0768, 0.0801, 0.0817, 0.0756,
        0.0772, 0.1585]) 
Epoch:1 | Score tensor([0.0782, 0.0795, 0.0760, 0.0806, 0.0783, 0.0773, 0.0809, 0.0816, 0.0775,
        0.0782, 0.1601]) 
Epoch:1 | Score tensor([0.0782, 0.0795, 0.0760, 0.0806, 0.0783, 0.0773, 0.0809, 0.0816, 0.0775,
        0.0782, 0.1601]) 
Epoch:1 | Score tensor([0.0768, 0.0785, 0.0740, 0.0779, 0.0770, 0.0750, 0.0789, 0.0803, 0.0760,
        0.0764, 0.1577]) 
Epoch:1 | Score tensor([0.0768, 0.0785, 0.0740, 0.0779, 0.0770, 0.0750, 0.0789, 0.0803, 0.0760,
        0.0764, 0.1577]) 
Epoch:1 | Score tensor([0.0755, 0.0759, 0.0731, 0.0765, 0.0756, 0.0736, 0.0774, 0.0794, 0.0743,
        0.0742, 0.1551]) 
Epoch:1 | Score tensor([

Epoch:1 | Score tensor([ 0.0814,  0.0758,  0.0784,  0.0775,  0.1025,  0.0724,  0.0770,  0.0953,
         0.0853,  0.0783,  0.1689, -0.3108]) 
Epoch:1 | Score tensor([ 0.0814,  0.0758,  0.0784,  0.0775,  0.1025,  0.0724,  0.0770,  0.0953,
         0.0853,  0.0783,  0.1689, -0.3108]) 
Epoch:1 | Score tensor([ 0.0830,  0.0795,  0.0799,  0.0765,  0.1004,  0.0757,  0.0821,  0.0977,
         0.0856,  0.0789,  0.1702, -0.3084]) 
Epoch:1 | Score tensor([ 0.0830,  0.0795,  0.0799,  0.0765,  0.1004,  0.0757,  0.0821,  0.0977,
         0.0856,  0.0789,  0.1702, -0.3084]) 
Epoch:1 | Score tensor([ 0.0824,  0.0856,  0.0794,  0.0774,  0.1020,  0.0769,  0.0855,  0.0978,
         0.0843,  0.0810,  0.1713, -0.3102]) 
Epoch:1 | Score tensor([ 0.0824,  0.0856,  0.0794,  0.0774,  0.1020,  0.0769,  0.0855,  0.0978,
         0.0843,  0.0810,  0.1713, -0.3102]) 
Epoch:1 | Score tensor([ 0.0775,  0.0786,  0.0747,  0.0739,  0.0935,  0.0671,  0.0845,  0.0867,
         0.0764,  0.0725,  0.1590, -0.2731]) 
Epoch:

Epoch:1 | Score tensor([ 0.0855,  0.0809,  0.0798,  0.0819,  0.0819,  0.0795,  0.0908,  0.0924,
         0.0793,  0.0728,  0.1627, -0.2673]) 
Epoch:1 | Score tensor([ 0.0863,  0.0807,  0.0817,  0.0836,  0.0814,  0.0817,  0.0912,  0.0933,
         0.0801,  0.0743,  0.1647, -0.2740]) 
Epoch:1 | Score tensor([ 0.0863,  0.0807,  0.0817,  0.0836,  0.0814,  0.0817,  0.0912,  0.0933,
         0.0801,  0.0743,  0.1647, -0.2740]) 
Epoch:1 | Score tensor([ 0.0863,  0.0807,  0.0817,  0.0836,  0.0814,  0.0817,  0.0912,  0.0933,
         0.0801,  0.0743,  0.1647, -0.2740]) 
Epoch:1 | Score tensor([ 0.0884,  0.0835,  0.0837,  0.0833,  0.0819,  0.0828,  0.0929,  0.0947,
         0.0808,  0.0760,  0.1673, -0.2824]) 
Epoch:1 | Score tensor([ 0.0884,  0.0835,  0.0837,  0.0833,  0.0819,  0.0828,  0.0929,  0.0947,
         0.0808,  0.0760,  0.1673, -0.2824]) 
Epoch:1 | Score tensor([ 0.0916,  0.0846,  0.0877,  0.0860,  0.0839,  0.0871,  0.0951,  0.0965,
         0.0840,  0.0787,  0.1728, -0.3007]) 
Epoch:

...

Epoch:7 | Score tensor([    8.4905,     7.4555,     7.7295,     7.9418,     7.9642,     7.7255,
            8.2127,     7.9504,     7.4583,     7.8144,    15.4723, -1397.0419]) 
Epoch:7 | Score tensor([    8.4905,     7.4555,     7.7295,     7.9418,     7.9642,     7.7255,
            8.2127,     7.9504,     7.4583,     7.8144,    15.4723, -1397.0419]) 
Epoch:7 | Score tensor([    8.0553,     7.0671,     7.3281,     7.5417,     7.5638,     7.3272,
            7.7914,     7.5429,     7.0688,     7.4180,    14.6741, -1338.3276]) 
Epoch:7 | Score tensor([    8.0553,     7.0671,     7.3281,     7.5417,     7.5638,     7.3272,
            7.7914,     7.5429,     7.0688,     7.4180,    14.6741, -1338.3276]) 
Epoch:7 | Score tensor([    7.7194,     6.7661,     7.0200,     7.2293,     7.2461,     7.0224,
            7.4665,     7.2207,     6.7682,     7.1072,    14.0560, -1284.6201]) 
Epoch:7 | Score tensor([    7.7194,     6.7661,     7.0200,     7.2293,     7.2461,     7.0224,
            7.

Epoch:8 | Score tensor([   4.8887,    4.9399,    5.1522,    4.9929,    4.8614,    4.9881,
           4.9089,    5.2588,    4.9937,    4.5599,    9.6696, -348.7157]) 
Epoch:8 | Score tensor([   4.8887,    4.9399,    5.1522,    4.9929,    4.8614,    4.9881,
           4.9089,    5.2588,    4.9937,    4.5599,    9.6696, -348.7157]) 
Epoch:8 | Score tensor([   4.4790,    4.5157,    4.7338,    4.5758,    4.4550,    4.5673,
           4.4987,    4.8170,    4.5758,    4.1694,    8.8586, -319.5953]) 
Epoch:8 | Score tensor([   4.4790,    4.5157,    4.7338,    4.5758,    4.4550,    4.5673,
           4.4987,    4.8170,    4.5758,    4.1694,    8.8586, -319.5953]) 
Epoch:8 | Score tensor([   4.1225,    4.1583,    4.3642,    4.2143,    4.1031,    4.2113,
           4.1547,    4.4385,    4.2122,    3.8337,    8.1589, -294.9165]) 
Epoch:8 | Score tensor([   4.1225,    4.1583,    4.3642,    4.2143,    4.1031,    4.2113,
           4.1547,    4.4385,    4.2122,    3.8337,    8.1589, -294.9165]) 
Epoc

Epoch:8 | Score tensor([   4.7152,    5.0019,    5.1809,    4.8051,    5.1544,    5.2620,
           4.9572,    4.9164,    5.2921,    4.8150,    9.7947, -914.9352]) 
Epoch:8 | Score tensor([   4.7152,    5.0019,    5.1809,    4.8051,    5.1544,    5.2620,
           4.9572,    4.9164,    5.2921,    4.8150,    9.7947, -914.9352]) 
Epoch:8 | Score tensor([   4.7152,    5.0019,    5.1809,    4.8051,    5.1544,    5.2620,
           4.9572,    4.9164,    5.2921,    4.8150,    9.7947, -914.9352]) 
 | Loss 1711.2005 | ||: 100%|██████████| 34/34 [00:00<00:00, 42.61it/s]
Epoch:9 | Score tensor([  2.9428,   2.3136,   2.8216,   2.5150,   2.3893,   2.4298,   2.6278,
          2.4950,   2.4698,   2.3887,   4.9663, -40.5767]) 
Epoch:9 | Score tensor([  2.9428,   2.3136,   2.8216,   2.5150,   2.3893,   2.4298,   2.6278,
          2.4950,   2.4698,   2.3887,   4.9663, -40.5767]) 
Epoch:9 | Score tensor([  2.9428,   2.3136,   2.8216,   2.5150,   2.3893,   2.4298,   2.6278,
          2.4950,   2.4698, 

Epoch:9 | Score tensor([  0.4771,   0.4453,   0.5050,   0.4460,   0.4332,   0.4317,   0.4562,
          0.4731,   0.4206,   0.4765,   0.8365, -22.6679]) 
Epoch:9 | Score tensor([  0.4335,   0.4104,   0.4702,   0.4140,   0.4041,   0.3961,   0.4178,
          0.4337,   0.3875,   0.4445,   0.7668, -21.5881]) 
Epoch:9 | Score tensor([  0.4335,   0.4104,   0.4702,   0.4140,   0.4041,   0.3961,   0.4178,
          0.4337,   0.3875,   0.4445,   0.7668, -21.5881]) 
Epoch:9 | Score tensor([  0.4621,   0.4371,   0.4895,   0.4462,   0.4259,   0.4163,   0.4462,
          0.4541,   0.4045,   0.4632,   0.8133, -21.2603]) 
Epoch:9 | Score tensor([  0.4621,   0.4371,   0.4895,   0.4462,   0.4259,   0.4163,   0.4462,
          0.4541,   0.4045,   0.4632,   0.8133, -21.2603]) 
Epoch:9 | Score tensor([  0.4077,   0.3850,   0.4443,   0.3944,   0.3741,   0.3711,   0.3972,
          0.3996,   0.3590,   0.4158,   0.7168, -20.1982]) 
Epoch:9 | Score tensor([  0.4077,   0.3850,   0.4443,   0.3944,   0.3741,   

Epoch:10 | Score tensor([ 0.0178,  0.0058,  0.0253,  0.0908,  0.0506,  0.0491,  0.0433,  0.0697,
        -0.0050,  0.0375,  0.0452,  0.2744]) 
Epoch:10 | Score tensor([ 0.0178,  0.0058,  0.0253,  0.0908,  0.0506,  0.0491,  0.0433,  0.0697,
        -0.0050,  0.0375,  0.0452,  0.2744]) 
Epoch:10 | Score tensor([0.0396, 0.0339, 0.0430, 0.1073, 0.0587, 0.0623, 0.0580, 0.0792, 0.0093,
        0.0664, 0.0778, 0.1370]) 
Epoch:10 | Score tensor([0.0396, 0.0339, 0.0430, 0.1073, 0.0587, 0.0623, 0.0580, 0.0792, 0.0093,
        0.0664, 0.0778, 0.1370]) 
Epoch:10 | Score tensor([0.0267, 0.0266, 0.0285, 0.0947, 0.0541, 0.0528, 0.0507, 0.0635, 0.0003,
        0.0581, 0.0556, 0.2491]) 
Epoch:10 | Score tensor([0.0267, 0.0266, 0.0285, 0.0947, 0.0541, 0.0528, 0.0507, 0.0635, 0.0003,
        0.0581, 0.0556, 0.2491]) 
Epoch:10 | Score tensor([0.0248, 0.0255, 0.0329, 0.0903, 0.0527, 0.0546, 0.0532, 0.0736, 0.0089,
        0.0547, 0.0571, 0.2512]) 
Epoch:10 | Score tensor([0.0248, 0.0255, 0.0329, 0.0903, 0.

In [5]:
print("unknown mse: {}".format(unknown_param_mse))

unknown mse: 0.45998963713645935


In [6]:
unknown_params

tensor([[0.2920],
        [0.2083],
        [0.1558],
        [0.0194],
        [0.1710],
        [0.1834],
        [0.2076],
        [0.1835],
        [0.4539],
        [0.1624],
        [0.6774]])

In [7]:
ols_param_mse

2.0813172

In [ ]:
store.close()
reader = CollectionReader(STORE_PATH)
results = reader.df(STORE_TABLE_NAME)
results.head()

  0%|          | 0/4 [00:00<?, ?it/s]

In [9]:
len(results)

TypeError: object of type 'LinearUnknownVariance' has no len()

In [ ]:
known, unknown, ols = results[['known_param_mse', 'var']], results[['unknown_param_mse', 'var']], results[['ols_param_mse', 'var']]

In [ ]:
# plot results for regression parameter MSE
sns.lineplot(data=unknown, x='var', y='unknown_param_mse', label='unknown', color='blue')
sns.lineplot(data=known, x='var', y='known_param_mse', label='known', color='purple')
ax = sns.lineplot(data=ols, x='var', y='ols_param_mse', label='ols', color='red')
ax.set(xlabel='Noise Variance', ylabel='Squared Error')
plt.show()
# close reader
reader.close()

In [21]:
store.close()

In [ ]:
print("hi")

In [10]:
store.close()

torch.Size([10, 10000])